# NLP Ansatz, um Posts zu Differenzieren

Folgend werden Posts geclustert und unterschiedlichen Kategorien zugewiesen.

### Requirements

In [1]:
# Final Variablen
API_KEY_SPOONACULAR = "146fe79a630b4afb8306a2985bedc64c"
API_KEY_SPOONACULAR = "6e0a91c81983423792b327bc4b78bb1d"

API_KEY_REDDIT = "KAJ2xGzCOlH3hyTRADNdSBDuTBORvw"
API_CLIENTID_REDDIT = "tSemvcUBRFCTYQ"

USERNAME_REDDIT = "Short-Arrival7632"
PW_REDDIT = "cookies123"

In [2]:
# KONFIGURATION

LIMIT_REDDIT_POSTS = 100
LIMIT_SPOONACULAR_RECIPES = 100
REDDIT_SAVE_INTERVALL = 10 # Minuten

In [3]:
# Auf Windows ausführen (einmalig)
# !pip install praw
# !pip install pandas
# !pip install matplotlib
# !pip install mysql-connector-python

In [4]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as pdt
import mysql.connector
import re

### Connect to Database

In [5]:
db_connection = mysql.connector.connect(
  host= "wp.jagi.wtf",
  user= "redditu",
  password= "redditMaster",
  database="reddit"
  )


In [6]:
cursor = db_connection.cursor()

### Get Data from Subreddit DB

In [7]:
df = pd.read_sql("SELECT * FROM reddit", db_connection)

# NLP STARTS HERE

In [8]:
df = df.drop(columns=['hour_created', 'time_created', 'day_created', 'author',
       'ups', 'downs', 'num_comments', 'text', 'thumbnail', 'url', 'curr_time'])

df.head()

,id,title
0,101,[PRO/CHEF] Texas Barbecue Platter
1,102,[homemade] Whole lemon cake doughnuts with whi...
2,103,[Homemade] Calamari
3,104,[homemade] raspberry cream cake covered in Legos
4,105,[homemade] “yogurt eggs and apple fries”


In [9]:
# Auf Windows ausführen (einmalig)
# !pip install tmtoolkit
# !pip install nltk
# !pip install pyLDAvis
# !pip install
# !pip install tqdm
# !pip install spacy==2.3.5
# !python -m spacy download en_core_web_sm # sonst googlen (kommt auf das env setup an)

# one time run to make this notebook work
# import nltk
# nltk.download('averaged_perceptron_tagger')

In [10]:
import os
import sys

# maths
import numpy as np

# regular expressions to manipulate text
import re
import string

# NLP
from tmtoolkit.preprocess import TMPreproc
from tmtoolkit.corpus import Corpus
# from tmtoolkit.defaults import language
from tmtoolkit.topicmod.tm_lda import evaluate_topic_models
from tmtoolkit.topicmod.evaluate import results_by_parameter
from tmtoolkit.topicmod.visualize import plot_eval_results
from tmtoolkit.topicmod.model_io import save_ldamodel_summary_to_excel
from tmtoolkit.topicmod.visualize import parameters_for_ldavis
from tmtoolkit.topicmod.model_io import save_ldamodel_to_pickle
from tmtoolkit.topicmod.model_io import ldamodel_top_topic_words
from tmtoolkit.topicmod.model_stats import marginal_topic_distrib
from tmtoolkit.bow.bow_stats import doc_lengths
from tmtoolkit.topicmod.model_stats import generate_topic_labels_from_top_words

import en_core_web_sm

import pyLDAvis

import logging
import warnings

# progress bars
from tqdm import tqdm

### Cleaning

In [11]:
## simple preprocessing

## Notiz an mich selbst 03.04.2020
## Nachschauen, wie regular expressions jetzt funktionieren
## 3.5. falscher Iterator

print('amount of data rows before: {}'.format(len(df)))


# for idx in tqdm(df.index):
#     text = df.loc[idx,'title']
# #     # remove url's
# #     # from https://docs.microsoft.com/en-us/previous-versions/msp-n-p/ff650303(v=pandp.10)?redirectedfrom=MSDN
# #     text = re.sub("^(ht|f)tp(s?)\:\/\/[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%\$#_]*)?$",                   '', text)
# #     # mail adresses
# #     text = re.sub('\S+@\S+', '', text)
# #     # web pages
# #     text = re.sub('\S+.com', '', text)
# #     # digits
# #     text = re.sub('\w*\d\w*', '', text)
# #     # non-sensical text
# #     text = re.sub('\n', ' ', text)
# #     text = re.sub('\r', ' ', text)
# #     text = re.sub('\\xa0', ' ', text)
#     # german letters
#     text = re.sub('ö','oe',text)
#     text = re.sub('Ö','Oe',text)
#     text = re.sub('ü','ue',text)
#     text = re.sub('Ü','Ue',text)
#     text = re.sub('ä','ae',text)
#     text = re.sub('Ä','Ae',text)
#     text = re.sub('ß','ss',text)
#     # removing extra white spaces
#     text = " ".join(text.split())
#     df.loc[idx,'title'] = text
    
df = df.dropna()
print('amount of data rows after: {}'.format(len(df)))

amount of data rows before: 357200
amount of data rows after: 357200


In [12]:
df.head()

,id,title
0,101,[PRO/CHEF] Texas Barbecue Platter
1,102,[homemade] Whole lemon cake doughnuts with whi...
2,103,[Homemade] Calamari
3,104,[homemade] raspberry cream cake covered in Legos
4,105,[homemade] “yogurt eggs and apple fries”


### Training

In [13]:
## globals

parent_path = os.path.dirname(os.getcwd())
language = 'english'

# suppress the "INFO" messages and warnings from lda
logger = logging.getLogger('lda')
logger.addHandler(logging.NullHandler())
logger.propagate = False

warnings.filterwarnings('ignore')

stop = ['homemade']

In [14]:
## formatting for tmtoolkit
d = df.set_index('id')['title'].to_dict()

In [15]:
#tmtoolkit preprocessing
## multiple preprocessing steps to be computed and compared against each other

corpus = Corpus(docs=d)
print(corpus)
#corpus = corpus.sample(50)

Corpus with 357200 documents


In [16]:
pp0 = TMPreproc(corpus, language='en')
del corpus

old_vocab = pp0.vocabulary
print('vocabulary size before processing: {}'.format(pp0.vocabulary_size))

vocabulary size before processing: 5965


In [17]:
## general simple preprocessing

pp0.pos_tag().lemmatize().tokens_to_lowercase().remove_special_chars_in_tokens()

general_vocab = pp0.vocabulary

print('vocabulary size after processing: {}'.format(pp0.vocabulary_size))
print('vocabulary, that was cut during this step:')
print(set(old_vocab) - set(general_vocab))
del old_vocab

vocabulary size after processing: 4250
vocabulary, that was cut during this step:
{'Sneaking', 'Drop', 'Dim', 'Focaccia', 'Mushrooms', 'Canadian', 'Marsala', 'Linguini', 'Ravioli', 'Sandwiche', 'Oxtail', 'spirals', 'Granola', 'Autumn', '-', 'Here', 'Purple', 'layers', 'Kreme', 'Big', 'Beef', 'Low', 'Erh', 'Pigs', 'toppers', 'Makhni', 'Shumai', 'Doughnuts', 'Mutter', 'Spice', 'Avacado', 'Adobo', 'Zucchini', 'Preserve', 'Feta', 'Granny', 'Heo', 'Toasted', 'Mutton', 'Pretzel', 'Argentinian', 'Smoothie', 'Wonton', 'Chip', 'Parfait', 'NACHOES', 'Mane', 'PF', 'forgotten', 'Naked', 'Out', 'STEAK', 'Korean/', 'West', 'Muhalbia', 'KING', 'Angel', 'Greek', 'Artisan', 'Avocado', 'giants', 'Portal', 'Donuts', 'Meatball', 'humans', 'Tape', 'Jello', 'Open', 'Herring', 'Nutella', 'picked', 'Served', 'Chargrilled', 'Portland', 'Crème', 'Fritters', 'Omelette', 'Jewelry', 'Hickory', 'bits', 'Aburi', 'Cross', 'Alta', 'Arabic', 'Lorraine', 'Uncured', 'Josh', 'Toast', 'Packet', 'French', 'Jalapenos', 'Dogs

In [18]:
## specific preprocessing

pp_aggressive = pp_nouns = pp_noadditionalstopwords = pp_staticstopwords = pp_dynamicstopwords = pp0.copy()
doc_labels = np.array(pp0.doc_labels)
del pp0

pp_dynamicstopwords.clean_tokens(remove_numbers=True, remove_shorter_than=2) \
    .remove_common_tokens(df_threshold=0.9) \
    .remove_uncommon_tokens(df_threshold=0.1)

pp_nouns.filter_for_pos('N') \
    .add_stopwords(stop) \
    .clean_tokens(remove_numbers=True, remove_shorter_than=3) \
    .remove_common_tokens(df_threshold=0.85) \
    .remove_uncommon_tokens(df_threshold=0.05)

pp_aggressive.filter_for_pos('N') \
    .add_stopwords(stop) \
    .clean_tokens(remove_numbers=True, remove_shorter_than=3) \
    .remove_common_tokens(df_threshold=0.9) \
    .remove_uncommon_tokens(df_threshold=0.1)

<TMPreproc [357200 documents / en]>

In [19]:
#vocab
vocab_dynamicstopwords = np.array(pp_dynamicstopwords.vocabulary)
vocab_nouns = np.array(pp_nouns.vocabulary)
vocab_aggressive = np.array(pp_aggressive.vocabulary)

#dtm
dtm_dynamicstopwords = pp_dynamicstopwords.dtm
dtm_nouns = pp_nouns.dtm
dtm_aggressive = pp_aggressive.dtm
del pp_dynamicstopwords, pp_nouns, pp_aggressive

In [20]:
# set data and hyperparameters to be computed

dtms = {
    'model_dynamicstopwords': dtm_dynamicstopwords,
    'model_nouns': dtm_nouns,
    'model_aggressive': dtm_aggressive
}

# var_params = [{'n_topics': k, 'alpha': 1/k, 'eta': 0.028*k,'n_iter': 200*k} for k in range(3, 20)]
# var_paramsas = [{'n_topics': k, 'alpha': 1/k, 'eta': 0.028*k,'n_iter': 200*k} for k in range(20, 3, -1)]
# var_params.extend(var_paramsas)

var_params = [{'n_topics': k, 'alpha': 1/k, 'eta': 0.028*k} for k in range(3, 5)]

const_params = {
    'n_iter': 1000,
    'random_state': 69  # to make results reproducible
}

var_params

[{'n_topics': 3, 'alpha': 0.3333333333333333, 'eta': 0.084},
 {'n_topics': 4, 'alpha': 0.25, 'eta': 0.112}]

In [ ]:
## compute all topic models and plot evaluation results

for dtm in dtms.keys():
    dtm_name = dtm
    dtm = dtms[dtm]
    
    #print('current dtm: {}'.format(dtm))
    
    eval_results = evaluate_topic_models(dtm,
                                        varying_parameters=var_params,
                                        constant_parameters=const_params,
                                        return_models=False)

    print('plotting...')
    
    eval_results_by_topics = results_by_parameter(eval_results, 'n_topics')
    plot_eval_results(eval_results_by_topics, xaxislabel=dtm_name + ' | n_topics');
    

    eval_results_by_topics = results_by_parameter(eval_results, 'n_iter')
    plot_eval_results(eval_results_by_topics, xaxislabel=dtm_name + ' | n_iter');
    

    eval_results_by_topics = results_by_parameter(eval_results, 'alpha')
    plot_eval_results(eval_results_by_topics, xaxislabel=dtm_name + ' | alpha');
    

    eval_results_by_topics = results_by_parameter(eval_results, 'eta')
    plot_eval_results(eval_results_by_topics, xaxislabel=dtm_name + ' | eta');

    print('{} done - waiting for plots'.format(dtm_name))

### Model Selection

In [ ]:
# ## best parameter sets for our different dtms 
# '''

# --- notes ---
# alpha: per document topic distribution (higher)
# eta: per topic word distribution (lower)

# '''
# # select dtm for following steps 
# dtm = dtms['model_staticstopwords']
# vocab = vocab_staticstopwords
# print('selected model removed the following vocabulary:')
# print(set(general_vocab) - set(vocab))
# #del general_vocab, dtms

In [ ]:
# # recompute chosen model
# from tmtoolkit.topicmod.tm_lda import compute_models_parallel

# topicn = 10

# lda_params = {
#     'n_topics': topicn,
#     'n_iter': 3750,
#     'alpha': 0.1,
#     'eta': 0.5,
#     'random_state': 20200701
# }

# model = compute_models_parallel(dtm, constant_parameters=lda_params)
# model

In [ ]:
# ## select model from nested list

# model = model[0][1]
# model

In [ ]:
# ## naming topics
# ## most 'important' topics for the whole corpus determined by the marginal topic distribution
# ## many more options to have a look into topics

# doc_lengths = doc_lengths(dtm)
# topic_labels = generate_topic_labels_from_top_words(
#     model.topic_word_,
#     model.doc_topic_,
#     doc_lengths,
#     vocab,
#     lambda_=0.6
# )

# top_topic_word = ldamodel_top_topic_words(model.topic_word_,
#                                           vocab,
#                                           row_labels=topic_labels)

# marg_topic = marginal_topic_distrib(model.doc_topic_, doc_lengths)

# # np.argsort() gives ascending order, hence reverse via [::-1]
# print('most important topics: \n{}'.format(topic_labels[np.argsort(marg_topic)[::-1][:topicn]]))